In [1]:
import os
import secrets
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime as date
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.tools import FigureFactory as FF
from sklearn.linear_model import LinearRegression
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn import linear_model, tree, neighbors
from sklearn.svm import SVR


In [2]:
df_strava_combined = pd.read_csv(r'C:\Users\annab\Google Drive\Projects\strava_analysis\strava_analysis\data\combined_activities.csv')
df_strava_api = pd.read_csv(r'C:\Users\annab\Google Drive\Projects\strava_analysis\strava_analysis\data\new_activities_raw.csv')
df_strava_historic = pd.read_csv(r'C:\Users\annab\Google Drive\Projects\strava_analysis\strava_analysis\data\historic_activities_raw.csv')

In [9]:

def compare_time_frames(df):
    
    """[Takes two sets of times periods and compares results of the two time frames]

    Args:
        file_path ([str]): [path to csv]
        start_1 ([type]): [beginning of time_frame_1]
        end_1 ([type]): [end of time_frame_1]
        start_2 ([type]): [beginning of time_frame_2]
        end_2 ([type]): [end of time_frame_2]
    """
    # Run faster in NC or SoFlo?
    #file_path ='static/demo_data/combined_activities.csv'
    start_1 = '2019-07-01'
    end_1 = '2020-04-01'
    start_2 = '2020-10-03'
    end_2 = '2021-04-26'
    # reads in activities df
    #df = pd.read_csv(file_path)


    #Create new dataframe with only columns I care about
    cols = ['name', 'upload_id', 'type', 'distance', 'moving_time',   
            'average_speed', 'max_speed','total_elevation_gain',
            'activity_date'
        ]
    df = df[cols]


    # change avg speed
    #df.loc[df["average_speed"] == 0,'average_speed'] = (df["distance"]  / df["moving_time"])*1000
    run = df.loc[df['type'] == 'Run'] 
 
    # convert time strings
    start_1 = date.strptime(start_1, '%Y-%m-%d').date()
    end_1 = date.strptime(end_1, '%Y-%m-%d').date()
    start_2 = date.strptime(start_2, '%Y-%m-%d').date()
    end_2 = date.strptime(end_2, '%Y-%m-%d').date()

    run['activity_date'] = pd.to_datetime(run['activity_date']).dt.date

    # chunk time frames

    # create column that is T if in t1 and F all else
    run['time_frame_1'] = np.where( (run['activity_date'] > start_1) & (run['activity_date'] < end_1), 'T', 'F')
    # get rows that fall in t1 column 
    time_frame_1 = run.loc[run['time_frame_1'] == 'T']


    
    # create column that is F if in t2 and F all else
    run['time_frame_2'] = np.where( (run['activity_date'] > start_2) & (run['activity_date'] < end_2), 'T', 'F')
    # get rows that fall in t2 column
    time_frame_2 = run.loc[run['time_frame_2'] == 'T']

    # get speed for each time period and also converts (time is in seconds, distance is in meters.
    #  m/s to mph 1 m/s to 2.237 mph)
    t1_speed = round(time_frame_1['average_speed'].mean() * 2.237, 2)
    t2_speed = round(time_frame_2['average_speed'].mean() * 2.237, 2)
    t1_max_speed = round(time_frame_1['max_speed'].mean()* 2.237, 2)
    t2_max_speed = round(time_frame_2['max_speed'].mean()* 2.237, 2)

    print("Average T1 Speed: " + str(t1_speed) + " | Average T1 Max Speed: " + str(t1_max_speed) + '\n'
        + "Average T2 Speed: " + str(t2_speed) + " | Average T2 Max Speed: " + str(t2_max_speed))

    percent_increase_average = round((t2_speed - t1_speed) * 100 / t1_speed,2)
    print("Percent increase average speed:",percent_increase_average)

    percent_increase_average_max = round((t2_max_speed - t1_max_speed) * 100 / t1_max_speed,2)
    print("Percent increase max speed:",percent_increase_average_max)

    #sns.scatterplot(x='total_elevation_gain', y = 'average_speed', data = run).set_title("Speed vs Elevation Gain")
    #sns.regplot(x='total_elevation_gain', y = 'average_speed', data = run).set_title("Speed vs Elevation Gain")
    # saves image
    #plt.show()
    y1 = [t1_speed, t1_max_speed]
    y2 = [t2_speed,t2_max_speed]
    #df_compare = pd.DataFrame(list(zip(l1,l2)), columns = ['Average Speed','Max Speed'])

    x_title=['Average Speed', 'Max Speed']

    fig = go.Figure(data=[
        go.Bar(name='Mountainous North Carolina', x=x_title, y=y1),
        go.Bar(name='Hot & Humid Florida', x=x_title, y=y2)
    ])
    # Change the bar mode
    fig.update_layout(barmode='group')
    
    plot_json = json.dumps(fig, cls = plotly.utils.PlotlyJSONEncoder)
    with open('graph_1_strava.txt', 'w') as outfile:
        json.dump(plot_json, outfile)

    select_time = pd.concat([time_frame_2,time_frame_1])
   
    # graph 2
    #df = px.data.tips()
    run = select_time[['total_elevation_gain','average_speed']]
   
    X = run.total_elevation_gain.values.reshape(-1, 1)

    model = LinearRegression()
    model.fit(X, run.average_speed)

    x_range = np.linspace(X.min(), X.max(), 100)
    y_range = model.predict(x_range.reshape(-1, 1))

    fig2 = px.scatter(run, x='total_elevation_gain', y='average_speed', opacity=0.65)
    fig2.add_traces(go.Scatter(x=x_range, y=y_range, name='Regression Fit'))
    
    plot_json2 = json.dumps(fig2, cls = plotly.utils.PlotlyJSONEncoder)
    with open('graph_2_strava.txt', 'w') as outfile:
        json.dump(plot_json2, outfile)



    ## three model graph


    # fig3 = train_and_display(run,model_selection)
    # plot_json3 = json.dumps(fig3, cls = plotly.utils.PlotlyJSONEncoder)

    return plot_json , plot_json2,run


def train_and_display(df, name):
    if name == None:
        name = 'knn'
    X = run.total_elevation_gain.values[:, None]
    X_train, X_test, y_train, y_test = train_test_split(
    X, run.average_speed, random_state=42)

    models = {'Regression': linear_model.LinearRegression,
        'DecisionTree': tree.DecisionTreeRegressor,
        'knn': neighbors.KNeighborsRegressor}
 
    model = models[name]()
    model.fit(X_train, y_train)

    x_range = np.linspace(X.min(), X.max(), 100)
    y_range = model.predict(x_range.reshape(-1, 1))

    fig = go.Figure([
        go.Scatter(x=X_train.squeeze(), y=y_train, 
                name='train', mode='markers'),
        go.Scatter(x=X_test.squeeze(), y=y_test, 
                name='test', mode='markers'),
        go.Scatter(x=x_range, y=y_range, 
                name='prediction')
        ])

    plot_json3 = json.dumps(fig, cls = plotly.utils.PlotlyJSONEncoder)
    with open('graph_3_strava_'+str(name)+'.txt', 'w') as outfile:
        json.dump(plot_json3, outfile)
    
   


    return plot_json3


def predict_kudos(df):

   # file_path ='static/demo_data/new_activities_clean.csv'
    #df = pd.read_csv(file_path)
        #Create new dataframe with only columns I care about
    cols = ['kudos_count', 'total_photo_count', 'type', 'distance', 'moving_time',   
            'average_speed', 'max_speed','total_elevation_gain',
            'id','comment_count','average_heartrate','average_temp'
        ]
    df = df[cols]

    mesh_size = .02
    margin = 0


    X = df[['max_speed', 'distance']]
    y = df['kudos_count']
    
    print(df)
    print(X)
    print(y)
    # df = px.data.iris()

    # X = df[['sepal_width', 'sepal_length']]
    # y = df['petal_width']
    # print(df)
    # print(X)
    # print(y)

    # Condition the model on sepal width and length, predict the petal width
    model = SVR(C=1.)
    model.fit(X, y)

    # Create a mesh grid on which we will run our model
    x_min, x_max = X.max_speed.min() - margin, X.max_speed.max() + margin
    y_min, y_max = X.distance.min() - margin, X.distance.max() + margin
    xrange = np.arange(x_min, x_max, mesh_size)
    yrange = np.arange(y_min, y_max, mesh_size)
    xx, yy = np.meshgrid(xrange, yrange)

    # Run model
    pred = model.predict(np.c_[xx.ravel(), yy.ravel()])
    pred = pred.reshape(xx.shape)

    # Generate the plot
    fig = px.scatter_3d(df, x='max_speed', y='distance', z='kudos_count')
    fig.update_traces(marker=dict(size=5))
    fig.add_traces(go.Surface(x=xrange, y=yrange, z=pred, name='pred_surface'))
    
    

    plot_json4 = json.dumps(fig, cls = plotly.utils.PlotlyJSONEncoder)
    with open('graph_4_strava_'+str(name)+'.txt', 'w') as outfile:
        json.dump(plot_json4, outfile)

    return plot_json4

In [ ]:
plot_json , plot_json2,run = compare_time_frames(df_strava_combined)
train_and_display(run, name=None)
train_and_display(run, name='Regression')
train_and_display(run, name='DecisionTree')

predict_kudos(df_strava_api)


     kudos_count  total_photo_count  type  distance  moving_time  \
0             16                  0   Run    8057.5         2277   
1             15                  0   Run    4842.8         1378   
2             20                  0   Run    8060.6         2286   
3             17                  0   Run    4847.7         1348   
4             11                  0  Walk    6442.5         4662   
..           ...                ...   ...       ...          ...   
195           11                  0   Run    1622.2          543   
196           15                  0   Run    1618.1          399   
197           13                  0   Run    8060.0         2252   
198           11                  0   Run    5006.0         1328   
199           15                  0   Run    6450.7         1953   

     average_speed  max_speed  total_elevation_gain          id  \
0            3.539        4.9                  74.0  5369343429   
1            3.514        5.2                  37